# Model Prototype

## Imports

In [32]:
import os
import re

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.layers import Embedding
from tensorflow.keras import layers
from tensorflow.keras import losses

print("Version: ", tf.__version__)
print("GPU is ","available" if tf.config.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.4.1
GPU is  available


## Data Ingestion

In [2]:
parent_dir = os.path.dirname(os.getcwd())
print(parent_dir)
CSV_dir = os.path.join(parent_dir, 'ml/data/data.csv')
print(CSV_dir)

/text-classifier/Text-Classifer
/text-classifier/Text-Classifer/ml/data/data.csv


In [3]:
df = pd.read_csv(CSV_dir)
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1
...,...,...
49995,I thought this movie did a down right good job...,1
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",0
49997,I am a Catholic taught in parochial elementary...,0
49998,I'm going to have to disagree with the previou...,0


## Data Analysis

In [4]:
print(df['review'][20])

After the success of Die Hard and it's sequels it's no surprise really that in the 1990s, a glut of 'Die Hard on a .....' movies cashed in on the wrong guy, wrong place, wrong time concept. That is what they did with Cliffhanger, Die Hard on a mountain just in time to rescue Sly 'Stop or My Mom Will Shoot' Stallone's career.<br /><br />Cliffhanger is one big nit-pickers dream, especially to those who are expert at mountain climbing, base-jumping, aviation, facial expressions, acting skills. All in all it's full of excuses to dismiss the film as one overblown pile of junk. Stallone even managed to get out-acted by a horse! However, if you an forget all the nonsense, it's actually a very lovable and undeniably entertaining romp that delivers as plenty of thrills, and unintentionally, plenty of laughs.<br /><br />You've got to love John Lithgows sneery evilness, his tick every box band of baddies, and best of all, the permanently harassed and hapless 'turncoat' agent, Rex Linn as Travers.

In [5]:
print(df['sentiment'][20])

1


## Data Preprocessing

#### Remove HTML Tags

In [6]:
def filter_html(row):
    f = re.compile(r'<.*?>')
    f = f.sub('', row['review'])
    f = f.replace("\'", "")
    return f

In [7]:
df['review'] = df.apply(filter_html, axis=1)
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. The filming tec...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically theres a family where a little boy (...,0
4,"Petter Matteis ""Love in the Time of Money"" is ...",1
...,...,...
49995,I thought this movie did a down right good job...,1
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",0
49997,I am a Catholic taught in parochial elementary...,0
49998,Im going to have to disagree with the previous...,0


In [8]:
print(df['review'][0])

One of the other reviewers has mentioned that after watching just 1 Oz episode youll be hooked. They are right, as this is exactly what happened with me.The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.I would say the main appeal of the show is due to the fact that it goes where other shows woul

#### Train Test Split 

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    df['review'], df['sentiment'], test_size=0.30, random_state=42
)

In [10]:
X_train.head()

38094    As much as I love trains, I couldnt stomach th...
40624    This was a very good PPV, but like Wrestlemani...
49425    Not finding the right words is everybodys prob...
35734    Im really suprised this movie didnt get a high...
41708    Ill start by confessing that I tend to really ...
Name: review, dtype: object

#### Convert to numpy array

In [11]:
X_train_n = X_train.to_numpy()

In [12]:
X_train_n

array(['As much as I love trains, I couldnt stomach this movie. The premise that one could steal a locomotive and "drive" from Arkansas to Chicago without hitting another train along the way has to be right up there on the Impossible Plot lines hit board. Imagine two disgruntled NASA employees stealing the "crawler" that totes the shuttles to and fro and driving it to New York and you get the idea.Having said all that, its a nice try. Wilford Brimely is at his Quaker Oats best, and Levon Helm turns a good performance as his dimwitted but well-meaning sidekick. Bob Balaban is suitably wormy as the Corporate Guy, and the "little guy takes on Goliath" story gets another airing.',
       'This was a very good PPV, but like Wrestlemania XX some 14 years later, the WWE crammed so many matches on it, some of the matches were useless. Im not going to go through every match on the card because it would take forever to do.However major highlights included the HUGE pop for Demolition winning the 

In [13]:
X_test_n = X_test.to_numpy()
y_train_n = y_train.to_numpy()
y_test_n = y_test.to_numpy()

#### Text Vectorization 

In [14]:
MAX_FEATURES = 10000
MAX_LEN = 200
EMBEDDING_DIMS = 2

In [15]:
vectorize_layer = TextVectorization(
    max_tokens = MAX_FEATURES,
    output_mode = 'int',
    output_sequence_length = MAX_LEN,
)
vectorize_layer

In [16]:
vectorize_layer.adapt(X_train_n)

#### Embedding 

In [17]:
embedding_layer = Embedding(
    MAX_FEATURES,
    16,
    input_length=MAX_LEN,
)

In [33]:
model = tf.keras.models.Sequential([
    tf.keras.Input(shape=(1,),dtype=tf.string),
    vectorize_layer,
    embedding_layer,
    layers.Dropout(0.2),
    layers.GlobalAveragePooling1D(),
    layers.Dropout(0.2),
    layers.Dense(1)
])

In [36]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 200)               0         
_________________________________________________________________
embedding (Embedding)        (None, 200, 16)           160000    
_________________________________________________________________
dropout_4 (Dropout)          (None, 200, 16)           0         
_________________________________________________________________
global_average_pooling1d_2 ( (None, 16)                0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 160,017
Trainable params: 160,017
Non-trainable params: 0
________________________________________________

In [34]:
model.compile(
    loss=losses.BinaryCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=tf.metrics.BinaryAccuracy(threshold=0.0)
)

## Model Training 

In [37]:
EPOCHS = 10

In [39]:
history = model.fit(
    X_train_n,
    validation_data=X_test_n,
    epochs=EPOCHS
)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [22]:
processed_X = model.predict(X_test_n)

In [23]:
processed_X.shape

(15000, 1)

In [25]:
processed_X[0]

array([-0.01522538], dtype=float32)

## Data Analysis 

In [26]:
processed_X_df = pd.DataFrame(processed_X)
processed_X_df.head()

,0
0,-0.015225
1,-0.001882
2,-0.027991
3,-0.038306
4,-0.001594


In [30]:
processed_X_df.describe()

,0
count,15000.000000
mean,-0.015394
std,0.014331
min,-0.060909
25%,-0.025453
50%,-0.011591
75%,-0.003336
max,0.009484


1.0